In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn 
import seaborn as sns

%matplotlib inline

C:\Users\Lenovo\anaconda3\Lib\site-packages\IPython\core\pylabtools.py:77: DeprecationWarning: backends is deprecated since IPython 8.24, backends are managed in matplotlib and can be externally registered.
  warnings.warn(


# Assosiation Rule Data Mining

Association Rule dalam data mining adalah teknik untuk menemukan pola atau hubungan antar item dalam dataset besar. Tujuan utamanya adalah mengidentifikasi item-item yang sering muncul bersama-sama.

## Prepare Dataset

In [96]:
df = pd.read_csv("dataset.csv")
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,NaN,0


In [97]:
# Cek missing Value
df.isna().sum()

id                       0
gender                   0
age                      0
hypertension             0
heart_disease            0
ever_married             0
work_type                0
Residence_type           0
avg_glucose_level        0
bmi                   1462
smoking_status       13292
stroke                   0
dtype: int64

In [98]:
len(df)

43400

In [99]:
# hapus baris yang memiliki missing values
df.dropna(inplace=True)

In [100]:
df.isna().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [101]:
# cek unique value tiap kolom
df.nunique()

id                   29072
gender                   3
age                     73
hypertension             2
heart_disease            2
ever_married             2
work_type                5
Residence_type           2
avg_glucose_level    11041
bmi                    516
smoking_status           3
stroke                   2
dtype: int64

In [102]:
df.gender.unique()

array(['Male', 'Female', 'Other'], dtype=object)

In [103]:
# Hanya pilih df yang gender-nya bukan "Other"
df = df.loc[df['gender'] != 'Other']

In [104]:
df.gender.unique()

array(['Male', 'Female'], dtype=object)

In [105]:
# Melihat kolom dengan jumlah nilai unik <= 3
columns_few_unique = [col for col in df.columns if df[col].nunique() <= 3]

columns_few_unique

['gender',
 'hypertension',
 'heart_disease',
 'ever_married',
 'Residence_type',
 'smoking_status',
 'stroke']

In [106]:
# Membuat data baru dari fitur yang sudah diseleksi
df_select = df[columns_few_unique]
df_select.head()

,gender,hypertension,heart_disease,ever_married,Residence_type,smoking_status,stroke
1,Male,1,0,Yes,Urban,never smoked,0
3,Female,0,0,Yes,Rural,formerly smoked,0
6,Female,0,0,Yes,Urban,formerly smoked,0
7,Female,0,1,Yes,Rural,never smoked,0
8,Female,0,0,Yes,Rural,smokes,0


## Preprocessing

In [108]:
# Buat salinan penuh dari df_select
df_select = df_select.copy()

# Lakukan encoding pada salinan
df_select['gender'] = label_encoder.fit_transform(df_select['gender'])
df_select['ever_married'] = label_encoder.fit_transform(df_select['ever_married'])
df_select['Residence_type'] = label_encoder.fit_transform(df_select['Residence_type'])

# Tampilkan hasil encoding
df_select.head()

,gender,hypertension,heart_disease,ever_married,Residence_type,smoking_status,stroke
1,1,1,0,1,1,never smoked,0
3,0,0,0,1,0,formerly smoked,0
6,0,0,0,1,1,formerly smoked,0
7,0,0,1,1,0,never smoked,0
8,0,0,0,1,0,smokes,0


In [109]:
df_select.drop('smoking_status', axis=1, inplace=True)

## Eksekusi assosiation rule

Kita akan menggunakan `Apriori` unutk melakukan assosiation rule pada dataset yang sudah kita olah di atas

**Siapkan Tools yang kita butuhkan**

In [110]:
!pip install mlxtend

### Apa itu apriori?

Apriori adalah algoritma yang digunakan dalam data mining untuk menemukan pola atau asosiasi dalam dataset, terutama dalam analisis keranjang belanja. Algoritma ini bekerja dengan cara mencari item-item yang sering muncul bersama dalam transaksi.

Proses kerjanya melibatkan dua langkah utama:

- **Menemukan Frequent Itemsets:** Algoritma ini mengidentifikasi itemsets yang muncul dalam transaksi lebih dari jumlah minimum yang ditentukan (support).

- **Menentukan Aturan Asosiasi:** Setelah menemukan itemsets yang sering muncul, algoritma ini membuat aturan asosiasi (misalnya, "Jika produk A dibeli, maka produk B juga cenderung dibeli") dengan menghitung nilai kepercayaan (confidence) dari setiap aturan.

Kelebihan dari algoritma Apriori adalah kemampuannya untuk menangani dataset besar dan kompleks. Namun, algoritma ini juga memiliki kekurangan, seperti kinerja yang lambat pada dataset yang sangat besar.

In [133]:
from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd

# Pastikan nilai lebih dari 0 menjadi True, yang lain menjadi False
df_association = df_select > 0

df_association.head()

,gender,hypertension,heart_disease,ever_married,Residence_type,stroke
1,True,True,False,True,True,False
3,False,False,False,True,False,False
6,False,False,False,True,True,False
7,False,False,True,True,False,False
8,False,False,False,True,False,False


In [151]:
# Menerapkan algoritma Apriori untuk menemukan frequent itemsets
frequent_itemsets = apriori(df_association, min_support=0.1, use_colnames=True)

# Menemukan association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

In [153]:
print("Frequent Itemsets:")
frequent_itemsets

Frequent Itemsets:


,support,itemsets
0,0.385790,(gender)
1,0.111509,(hypertension)
2,0.746155,(ever_married)
3,0.502047,(Residence_type)
4,0.293136,"(ever_married, gender)"
5,0.195252,"(gender, Residence_type)"
6,0.101118,"(ever_married, hypertension)"
7,0.375675,"(ever_married, Residence_type)"
8,0.148185,"(ever_married, gender, Residence_type)"


# Analisis pada kolom `Frequent Itemsets`

**Frekuensi Kemunculan Fitur Tunggal:**

- ever_married (pernah menikah) adalah fitur yang paling sering muncul, dengan nilai support sebesar 0.746. Artinya, sekitar 74.6% dari data adalah individu yang pernah menikah.

- Residence_type (tipe tempat tinggal) memiliki nilai support sebesar 0.502, menunjukkan bahwa sekitar 50.2% data adalah individu dengan tipe tempat tinggal tertentu (misalnya, Rural atau Urban).

- gender memiliki support sebesar 0.385, artinya sekitar 38.5% data memiliki jenis kelamin tertentu.


**Frekuensi Kemunculan Kombinasi Fitur (Itemsets):**

- Kombinasi (ever_married, gender) memiliki nilai support 0.293, yang berarti sekitar 29.3% data adalah individu yang pernah menikah dengan jenis kelamin tertentu.

- Kombinasi (gender, Residence_type) memiliki nilai support 0.195, menunjukkan bahwa sekitar 19.5% data adalah individu dengan jenis kelamin tertentu dan tipe tempat tinggal tertentu.

- Kombinasi (ever_married, Residence_type) memiliki nilai support 0.376, artinya 37.6% dari data terdiri dari individu yang pernah menikah dan memiliki tipe tempat tinggal tertentu.

- Kombinasi (ever_married, gender, Residence_type) memiliki support 0.148, yang menunjukkan bahwa 14.8% dari data adalah individu dengan ketiga karakteristik ini (pernah menikah, memiliki jenis kelamin tertentu, dan tipe tempat tinggal tertentu).

In [163]:
print("Association Rules:")
rules

Association Rules:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(gender),(ever_married),0.385790,0.746155,0.293136,0.759832,1.018330,0.005277,1.056948,0.029306
1,(hypertension),(ever_married),0.111509,0.746155,0.101118,0.906819,1.215322,0.017915,2.724209,0.199409
2,(Residence_type),(ever_married),0.502047,0.746155,0.375675,0.748287,1.002857,0.001070,1.008468,0.005721
3,"(gender, Residence_type)",(ever_married),0.195252,0.746155,0.148185,0.758943,1.017138,0.002497,1.053048,0.020937


# Analisis pada `rules`

Ambil contoh analisis pada : (gender, Residence_type) -> (ever_married)

- `Support: 0.148185`, yang berarti sekitar `14.8%` dari data memiliki kombinasi `gender`, `Residence_type`, dan `ever_married`.

- `Confidence: 0.758943`, menunjukkan bahwa ada peluang sekitar `75.89%` bahwa individu dengan kombinasi `gender` dan `Residence_type` tertentu juga `ever_married`.

- `Lift: 1.017138`, yang menunjukkan bahwa hubungan antara kombinasi `gender` dan `Residence_type` terhadap status `ever_married` juga lemah.

Kesimpulan: Kombinasi `gender` dan `Residence_type` tidak memiliki korelasi kuat dengan status `ever_married`.

# Terimakasih